In [1]:
from scipy import stats
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import TimeSeriesSplit
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

/anaconda3/envs/py37/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
train = pd.read_pickle('../src/make_data/data/030_train.pkl')
test = pd.read_pickle('../src/make_data/data/030_test.pkl')

In [3]:
train = train.drop(['isFraud','TransactionDT','month'],axis=1)
test = test.drop(['TransactionDT','month'],axis=1)

In [4]:
for f in tqdm(train.select_dtypes(include='category').columns.tolist() + train.select_dtypes(include='object').columns.tolist()):
    lbl = LabelEncoder()
    lbl.fit(list(train[f].values) + list(test[f].values))
    train[f] = lbl.transform(list(train[f].values))
    test[f] = lbl.transform(list(test[f].values))

100%|██████████| 41/41 [01:17<00:00,  1.04it/s]


In [5]:
train['target'] = 0
test['target'] = 1

In [6]:
train_test = pd.concat([train, test], axis =0)
train_test = train_test.sample(frac=1)
# target = train_test['target'].values

In [11]:
x = train_test.drop('target',axis=1)
y = train_test['target']

In [8]:
param = {'num_leaves': 50,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': 5,
         'learning_rate': 0.2,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 44,
         "metric": 'auc',
         "verbosity": -1}

In [13]:
kfold_splits = 5
folds = KFold(n_splits=kfold_splits,shuffle=True)
for fold_, (trn_idx, val_idx) in enumerate(folds.split(x.values, y.values)):
    print(val_idx)

[      7      11      14 ... 1097224 1097227 1097229]
[      1       3       4 ... 1097199 1097208 1097218]
[      0       9      10 ... 1097219 1097222 1097225]
[      5      20      21 ... 1097214 1097226 1097230]
[      2       8      12 ... 1097204 1097215 1097228]
